In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import time

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)




In [ ]:
batch_size = 32

LInputTarget = tf.keras.Input(dtype = tf.float32, shape = [256, 256, 3], name = 'Target')
with tf.name_scope("Encoder"):
    Lconv1_1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LInputTarget) 
    Lconv1_2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lconv1_1) 
    Lpool1 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(Lconv1_2)

    Lconv2_1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lpool1) 
    Lconv2_2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lconv2_1) 
    Lpool2 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(Lconv2_2)

    Lconv3_1 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lpool2) 
    Lconv3_2 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lconv3_1) 
    Lpool3 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(Lconv3_2)

    Lconv4_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lpool3) 
    Lconv4_2 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lconv4_1) 
    Lpool4 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(Lconv4_2)

    Lconv5_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lpool4) 
    Lconv5_2 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(Lconv5_1) 
    Lpool5 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(Lconv5_2)
    
    modelEncoder = tf.keras.Model(inputs=LInputTarget, outputs=Lpool5, name="Unet model")

In [ ]:
modelEncoder.summary()

In [ ]:
tf.keras.utils.plot_model(modelEncoder, "unet_encoder.png", show_shapes=True)

In [ ]:
LInputQuery  = tf.keras.Input(dtype = tf.float32, shape = [64, 64, 3], name = 'Query')
with tf.name_scope("Conditional"):
    LCconv1_1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LInputQuery) 
    LCconv1_2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LCconv1_1) 
    LCpool1 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(LCconv1_2)

    
    LCconv2_1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LCpool1) 
    LCconv2_2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LCconv2_1) 
    LCpool2 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(LCconv2_2)


    LCconv3_1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LCpool2) 
    LCpool3 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(LCconv3_1)

    
    LCconv4_1 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LCpool3) 
    LCpool4 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(LCconv4_1)

    
    LCconv5_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LCpool4)  
    LCpool5 = tf.keras.layers.MaxPool2D(pool_size=2, strides = 2, padding = 'SAME')(LCconv5_1)
    
    LCconv6_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 2, strides = 1, 
                   activation = tf.nn.relu)(LCpool5)
    LCoutput = LCconv6_1
    modelConditional = tf.keras.Model(inputs=LInputQuery, outputs=LCconv6_1, name="latent representation")

In [ ]:
modelConditional.summary()

In [ ]:
tf.keras.utils.plot_model(modelConditional, "unet_encoder.png", show_shapes=True)

In [ ]:
#LInputIntermittent  = tf.keras.Input(dtype = tf.float32, shape = [8, 8, 1024], name = 'Encoded intermittent')
with tf.name_scope("Transcoder"):
    LDtiling8x8 = tf.keras.layers.UpSampling2D(size=(8, 8))(LCoutput) # [1,1,512] -> [8,8,512]
    LDconcat1_1 = tf.keras.layers.Concatenate(axis=-1)([Lpool5, LDtiling8x8]) # -> [8,8,1024]
    LDdeconv1_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDconcat1_1) # [8,8,1024] -> [8,8,512]
    LDdeconv1_2 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDdeconv1_1)  # [8,8,512]  -> [8,8,512]
    LDupsam1 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size = 3,
                   strides=(2,2), padding='SAME', activation = tf.nn.relu)(LDdeconv1_2) # [8,8,512] -> [16,16,512]

    
    LDtiling16x16 = tf.keras.layers.UpSampling2D(size=(16, 16))(LCoutput) # [1,1,512] -> [16,16,512]
    LDconcat2_1 = tf.keras.layers.Concatenate(axis=-1)([Lpool4, LDtiling16x16]) # -> [16,16,1024]
    LDconv2 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 1, strides = 1, padding='SAME',
                   activation=tf.nn.relu)(LDconcat2_1) # [ 16, 16, 1024 ] -> [ 16, 16, 512 ]
    LDconcat2_2 = tf.keras.layers.Concatenate(axis=-1)([LDupsam1, LDconv2]) # -> [ 16, 16, 1024 ]
    LDdeconv2_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDconcat2_2) # [16,16,1024] -> [16,16,512]
    LDdeconv2_2 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDdeconv2_1)  # [16,16,256]  -> [16,16,512]
    LDupsam2 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size = 3,
                   strides=(2,2), padding='SAME', activation = tf.nn.relu)(LDdeconv2_2) # [16,16,512] -> [32,32,512]
    
    
    LDtiling32x32 = tf.keras.layers.UpSampling2D(size=(32, 32))(LCoutput) # [1,1,512] -> [32,32,512]
    LDconcat3_1 = tf.keras.layers.Concatenate(axis=-1)([Lpool3, LDtiling32x32]) # -> [32,32,1024]
    LDconv3 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 1, strides = 1, padding='SAME',
                   activation=tf.nn.relu)(LDconcat3_1) # -> [32, 32, 256]
    
    LDconcat3_2 = tf.keras.layers.Concatenate(axis=-1)([LDupsam2, LDconv3]) # -> [32,32,768]
    LDdeconv3_1 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDconcat3_2) # [32,32,768] -> [32,32,256]
    LDdeconv3_2 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDdeconv3_1)  # [32,32,256]  -> [32,32,256]
    LDupsam3 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size = 3,
                   strides=(2,2), padding='SAME', activation = tf.nn.relu)(LDdeconv3_2) # -> [64,64,256]
    
    
    LDtiling64x64 = tf.keras.layers.UpSampling2D(size=(64, 64))(LCoutput) # [1,1,512] -> [64,64,512]
    LDconcat4_1 = tf.keras.layers.Concatenate(axis=-1)([Lpool2, LDtiling64x64]) # -> [64,64,768]
    LDconv4 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 1, strides = 1, padding='SAME',
                   activation=tf.nn.relu)(LDconcat4_1) # -> [64, 64, 128]
    
    LDconcat4_2 = tf.keras.layers.Concatenate(axis=-1)([LDupsam3, LDconv4]) # -> [64,64,384]
    LDdeconv4_1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDconcat4_2) # [64,64,384] -> [64,64,128]
    LDdeconv4_2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDdeconv4_1)  # [64,64,128]  -> [64,64,128]
    LDupsam4 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size = 3,
                   strides=(2,2), padding='SAME', activation = tf.nn.relu)(LDdeconv4_2) # -> [128,128,128]
    
    
    LDtiling128x128 = tf.keras.layers.UpSampling2D(size=(128, 128))(LCoutput) # [1,1,512] -> [128,128,512]
    LDconcat5_1 = tf.keras.layers.Concatenate(axis=-1)([Lpool1, LDtiling128x128]) # -> [128,128,640]
    LDconv5 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 1, strides = 1, padding='SAME',
                   activation=tf.nn.relu)(LDconcat5_1) # -> [128, 128, 64]
    
    LDconcat5_2 = tf.keras.layers.Concatenate(axis=-1)([LDupsam4, LDconv5]) # -> [128,128,192]
    LDdeconv5_1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDconcat5_2) # [128,128,192] -> [128,128,64]
    LDdeconv5_2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu)(LDdeconv5_1)  # [128,128,64]  -> [128,128,64]
    LDupsam5 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size = 3,
                   strides=(2,2), padding='SAME', activation = tf.nn.relu)(LDdeconv5_2) # -> [256,256,64]
    LDoutput = tf.keras.layers.Conv2D(filters = 1, kernel_size = 3, strides = 1, padding='SAME', 
                   activation = tf.nn.relu, name="Output")(LDupsam5)  # [256,256,64]  -> [256,256,1]
    modelUnet = tf.keras.Model(inputs=[LInputTarget, LInputQuery], outputs=[LDoutput], name="Unet")

In [ ]:
modelUnet.summary()

In [ ]:
tf.keras.utils.plot_model(modelUnet, "unet_model.png", show_shapes=True)

# Przygotowanie danych

In [ ]:
import os
MASKS_PATH = '/qarr/studia/magister/datasets/FlickrLogos-v2/classes/masks/'
INPUT_PATH = '/qarr/studia/magister/datasets/FlickrLogos-v2/classes/jpg/'

classes = [o for o in os.listdir(INPUT_PATH) if os.path.isdir(INPUT_PATH + '/' + o)]
classes = [o for o in classes if o != 'no-logo']


In [ ]:
print(classes)

## Wczytywanie obrazków do pamięci

In [ ]:
images = dict()
targets = dict()
queries = dict()
start_time = time.time()

def rescale(nparray, scale=255.0):
    return np.array(nparray, dtype=np.float32)/scale

for c in classes:
    root_input = INPUT_PATH + '/' + c 
    root_masks = MASKS_PATH + '/' + c
    images[c] = list()
    targets[c] = list()
    queries[c] = list()
    
    for f in os.listdir(root_input):
        img = cv2.imread(f'{root_input}/{f}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(f'{root_masks}/{f}.mask.merged.png', cv2.IMREAD_GRAYSCALE)
        bboxes = []
        
        with open(f'{root_masks}/{f}.bboxes.txt') as csvfile:
            bboxread = csv.reader(csvfile, delimiter=' ')
            next(bboxread)
            for row in bboxread:
                bboxes.append(row)
                
        for bbox in bboxes:
            x,y,w,h = [int(i) for i in bbox]
            imgslice = img[y:y+h, x:x+w]
            imgslice = cv2.resize(imgslice, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
            queries[c].append(rescale(imgslice, 255.0))
            # Biore tylko pierwszy z dostepnych bbox na obrazku
            break 
            
        img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        mask = cv2.resize(mask, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    
        images[c].append(rescale(img, 255.0))
        targets[c].append(rescale(mask, 255.0))


end_time = time.time()

print(f'Time taken: {end_time-start_time} seconds')


## Generowanie listy labeli dla wczytanych obrazków

In [ ]:
images_labels = list()
targets_labels = list()
queries_labels = list()

for c in classes:
    images_labels += [classes.index(c)] * len(images[c])
    queries_labels += [classes.index(c)] * len(queries[c])
targets_labels = images_labels

## Wczytane przykłady obrazków

In [ ]:
for v in images.values():
    plt.imshow(v[0])
    break

In [ ]:
for v in targets.values():
    plt.imshow(v[0], cmap='gist_gray')
    break

In [ ]:
for v in queries.values():
    plt.imshow(v[0])
    break

# Tworzenie tymczasowych datasets

In [ ]:
sumimgs = sum(images.values(), [])
sumtargs = sum(targets.values(), [])
sumquers = sum(queries.values(), [])

In [ ]:
#images_dataset = tf.data.Dataset.from_tensor_slices((sumimgs, sumtargs, images_labels))

In [ ]:
#queries_dataset = tf.data.Dataset.from_tensor_slices((sumquers, queries_labels))

In [ ]:
for c in classes:
    print(f'{c:>12}: {len(images[c])} logos: {len(queries[c]):3<} pairs: {len(images[c])*len(queries[c])}')
print(f'{"total":<12}: {sum([len(images[c]) for c in classes])} logos: {sum([len(queries[c]) for c in classes])} pairs: {sum([len(images[c])*len(queries[c]) for c in classes])}')

In [ ]:
print(f"{32*(70*69)*(64*64*3+256*256*4)*8/1e9} GB vs {32*(70)*(64*64*3+256*256*4)*8/1e9} GB")

Gdyby brać pod uwagę każde logo z obrazka:
```
      adidas: 70 logos: 120 pairs: 8400
        aldi: 70 logos: 106 pairs: 7420
       apple: 70 logos: 76 pairs: 5320
       becks: 70 logos: 100 pairs: 7000
         bmw: 70 logos: 74 pairs: 5180
   carlsberg: 70 logos: 108 pairs: 7560
      chimay: 70 logos: 112 pairs: 7840
    cocacola: 70 logos: 130 pairs: 9100
      corona: 70 logos: 83 pairs: 5810
         dhl: 70 logos: 123 pairs: 8610
    erdinger: 70 logos: 105 pairs: 7350
        esso: 70 logos: 87 pairs: 6090
       fedex: 70 logos: 94 pairs: 6580
     ferrari: 70 logos: 73 pairs: 5110
        ford: 70 logos: 76 pairs: 5320
     fosters: 70 logos: 98 pairs: 6860
      google: 70 logos: 83 pairs: 5810
     guiness: 70 logos: 98 pairs: 6860
    heineken: 70 logos: 103 pairs: 7210
          hp: 70 logos: 112 pairs: 7840
       milka: 70 logos: 197 pairs: 13790
      nvidia: 70 logos: 114 pairs: 7980
    paulaner: 70 logos: 102 pairs: 7140
       pepsi: 70 logos: 178 pairs: 12460
 rittersport: 70 logos: 204 pairs: 14280
       shell: 70 logos: 96 pairs: 6720
      singha: 70 logos: 83 pairs: 5810
   starbucks: 70 logos: 95 pairs: 6650
stellaartois: 70 logos: 87 pairs: 6090
      texaco: 70 logos: 88 pairs: 6160
    tsingtao: 70 logos: 109 pairs: 7630
         ups: 70 logos: 90 pairs: 6300
total       : 2240 logos: 3404 pairs: 238280
```

# Generator danych

In [ ]:
def dataset_permutations_generator(batch_size):
    nclasses = len(classes)
    nlogos = sum([len(images[c]) for c in classes])//nclasses
    data_permutations = np.zeros((nclasses*nlogos*(nlogos-1), 3), dtype=np.int8)
    i = 0
    for c_i in range(nclasses):
        for n_i in range(nlogos):
            for l_i in range(nlogos):
                if n_i == l_i:
                    continue
                data_permutations[i] = (c_i, n_i, l_i)
                i += 1
    data_permutations = np.random.permutation(data_permutations)
    s = 0
    outimage = []
    outquery = []
    outtarget = []
    for class_number, image_number, query_number in data_permutations:
        c = classes[class_number]
        outimage.append(images[c][image_number])
        outquery.append(queries[c][query_number])
        outtarget.append(targets[c][image_number])
        s += 1
        if s >= batch_size:
            s = 0
            yield (np.reshape(outimage, (batch_size, 256, 256, 3)),
                   np.reshape(outquery, (batch_size, 64, 64, 3))
                  ), np.reshape(outtarget, (batch_size, 256, 256, 1))
            outimage = []
            outquery = []
            outtarget = [] 

## Test generatora

In [ ]:
testgen = dataset_permutations_generator(1)
tdat = next(testgen)

In [ ]:
plt.imshow(tdat[0][0][0])

In [ ]:
batch_size = 32
unetDataset = tf.data.Dataset.from_generator(dataset_permutations_generator,
                                             args=[batch_size],
                                             output_types=((tf.float32, tf.float32), tf.float32),
                                             output_shapes=(((batch_size, 256,256,3), (batch_size, 64,64,3)),
                                                          (batch_size, 256,256,1))
                                            )
#unetDataset = unetDataset.cache()

# Zestawienie modelu i uczenie - próba

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0004, momentum=0.9, nesterov=False, name="SGD") # weight decay 0.0005

modelUnet.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
#modelUnet.fit(unetDataset, epochs=1, steps_per_epoch=1)

In [ ]:
gen_inst = dataset_permutations_generator(32)
custom_batch = [next(gen_inst) for i in range(2)]

In [ ]:
def describe(x):
    try:
        return f'{x.shape}'
    except AttributeError:
        return f"{'[' + ', '.join([describe(q) for q in x]) + ']'}"


#custom_batch[0]
print(describe(custom_batch[0]))

In [ ]:
testDataset = tf.data.Dataset.from_tensors(custom_batch[0])
#testDataset = testDataset.batch(32)
#timgs, tquers, ttargs = ([a for a,b,c in custom_batch], [b for a,b,c in custom_batch], [c for a,b,c in custom_batch])

In [ ]:
img = custom_batch[0][0][0]
qry = custom_batch[0][0][1]

In [ ]:
plt.imshow(img[0])

In [ ]:
plt.imshow(qry[0])

In [ ]:
#modelUnet.predict({"Target":img, "Query":qry})

In [ ]:
a = [i for i in testDataset]
a = a[0]
a[0][1].shape

In [ ]:
modelUnet.fit(testDataset, epochs=1, steps_per_epoch=2)